In [15]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
%matplotlib inline

from skimage.feature import hog
from skimage import color, exposure

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.model_selection import train_test_split
import pickle
import model

from lesson_functions import *

## Process Data and Train a Classifier (SVC)

In [ ]:
images = glob.glob('training_set/*/*/*.png')
cars = []
notcars = []
for image in images:
    if 'non-vehicles' in image:
        notcars.append(image)
    else:
        cars.append(image)  

In [ ]:
image = mpimg.imread(notcars[0])
image_car = mpimg.imread(cars[0])

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(image_car)
ax2.imshow(image)

### Parameters

In [ ]:
color_space = 'HLS' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL"
spatial_feat = False # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

### Extracting features training sets

In [ ]:
t=time.time()
car_features = extract_features(cars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)

t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')

### Train SVC and X_scaler

In [ ]:
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t = time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

### Save trained models

In [ ]:
import pickle
pickle.dump(svc, open('svc.p', 'wb'))

In [ ]:
pickle.dump(X_scaler, open('X_scaler.p', 'wb'))

### Sliding Window and Detect Vehicles on Images

In [1]:
import detector
import imp
import pickle

In [76]:
imp.reload(detector)

<module 'detector' from '/Users/hchoi/Documents/Udacity_Self_Driving_Car/CarND-Vehicle-Detection/detector.py'>

In [77]:
d = detector.Detector()

In [78]:
d.svc = pickle.load(open('svc.p', 'rb'))

In [79]:
d.X_scaler = pickle.load(open('X_scaler.p', 'rb'))

In [80]:
def pipeline(original_img):
    d.new_frame()
    bboxes, _ = d.multiple_window_sizes_search(original_img)
    d.add_heat(bboxes)
    d.filter_false_positive()
    d.apply_threshold()
    d.apply_label()
    final_img = d.draw_labeled_bboxes(original_img)
    return final_img

In [81]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [82]:
d.reset()
d.threshold = 12

In [83]:
clip1 = VideoFileClip("project_video.mp4")
detected_clip = clip1.fl_image(pipeline) 
detected_output = 'project_video_output_2.mp4'
%time detected_clip.write_videofile(detected_output, audio=False)

[MoviePy] >>>> Building video project_video_output_2.mp4
[MoviePy] Writing video project_video_output_2.mp4


100%|█████████▉| 1260/1261 [18:09<00:00,  1.21it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output_2.mp4 

CPU times: user 17min 41s, sys: 41.2 s, total: 18min 22s
Wall time: 18min 9s
